In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dataframe1 = pd.read_csv('调整数据.csv')
dataframe2 = pd.read_csv('调整记录（分隔）.csv')
dataframe1,dataframe2

(        Logtime  Gage2_smooth  Gage2_Target
 0       17:50.2          2.63           2.9
 1       17:50.5          2.63           2.9
 2       17:50.8          2.63           2.9
 3       17:51.1          2.63           2.9
 4       17:51.4          3.31           2.9
 ...         ...           ...           ...
 277105  20:48.6          2.51           2.9
 277106  20:48.9          4.03           2.9
 277107  20:49.3          4.03           2.9
 277108  20:49.6          4.03           2.9
 277109  20:49.9          4.03           2.9
 
 [277110 rows x 3 columns],
           27\t146\t  \t0\t  12  21 10:18:01
 0         28\t146\t    0.0  12  21 10:18:03
 1         29\t146\t    0.0  12  21 10:18:05
 2         30\t146\t    0.0  12  21 10:18:07
 3         31\t146\t    0.0  12  21 10:18:09
 4         32\t146\t    0.0  12  21 10:18:11
 ...             ...    ...  ..          ...
 39868  39896\t151\t    0.0  12  22 10:29:58
 39869  39897\t151\t    0.0  12  22 10:30:00
 39870  39898\t151\t    0

In [3]:
water = dataframe1.iloc[:,-2]
target = dataframe1.iloc[:,-1]
time = dataframe2.iloc[:,-1]
adjust = dataframe2.iloc[:,-3]
water,target,time,adjust

(0         2.63
 1         2.63
 2         2.63
 3         2.63
 4         3.31
           ... 
 277105    2.51
 277106    4.03
 277107    4.03
 277108    4.03
 277109    4.03
 Name: Gage2_smooth, Length: 277110, dtype: float64,
 0         2.9
 1         2.9
 2         2.9
 3         2.9
 4         2.9
          ... 
 277105    2.9
 277106    2.9
 277107    2.9
 277108    2.9
 277109    2.9
 Name: Gage2_Target, Length: 277110, dtype: float64,
 0        21 10:18:03
 1        21 10:18:05
 2        21 10:18:07
 3        21 10:18:09
 4        21 10:18:11
             ...     
 39868    22 10:29:58
 39869    22 10:30:00
 39870    22 10:30:02
 39871    22 10:30:04
 39872    22 10:30:07
 Name: 21 10:18:01, Length: 39873, dtype: object,
 0        0.0
 1        0.0
 2        0.0
 3        0.0
 4        0.0
         ... 
 39868    0.0
 39869    0.0
 39870    0.0
 39871    0.0
 39872    0.0
 Name: \t0\t, Length: 39873, dtype: float64)

In [4]:
test_water = []
for i in range(0,277109,7):
    avg = water[i] + water[i+1] + water[i+2] + water[i+3] + water[i+4] + water[i+5] + water[i+6]
    test_water.append(avg/7)
len(test_water)

39587

In [5]:
test_target = []
for i in range(0,277109,7):
    avg = target[i] + target[i+1] + target[i+2] + target[i+3] + target[i+4] + target[i+5] + target[i+6]
    test_target.append(avg/7)
len(test_target)

39587

In [6]:
predict_water = np.zeros(29)
for i in range(0,39587):
    if test_water[i] <= test_target[i] + 0.05:
        predict_water = np.append(predict_water,0)
    else:
        predict_water = np.append(predict_water,1)

len(predict_water),predict_water[0:50]

(39616,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]))

In [7]:
predict_adjust = []
for i in range(30,39616):
    ratio = predict_water[i-30:i].sum()
    predict_adjust.append(ratio/30)
len(predict_adjust)

39586

In [8]:
import pyecharts.options as opts
from pyecharts.charts import Line, Grid
timeData = time[4000:14000]
adjustData = predict_adjust[6000:16000]
adjustdata = adjust[6400:16400]
water = test_water[6000:16000]
target = test_target[6000:16000]

In [9]:
(
    Line()
    .add_xaxis(xaxis_data=timeData)
    .add_yaxis(
        series_name="调整记录",
        y_axis=adjustData,
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        linestyle_opts=opts.LineStyleOpts(),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="调整预测",
        y_axis=adjustdata,
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        linestyle_opts=opts.LineStyleOpts(),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="监测数据",
        y_axis=water,
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        linestyle_opts=opts.LineStyleOpts(),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="监测指标",
        y_axis=target,
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        linestyle_opts=opts.LineStyleOpts(),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="比例",
            name_location="start",
            type_="value",
            max_=5,
            is_inverse=True,
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="调整记录",
            subtitle="永信",
            pos_left="center",
            pos_top="top",
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        legend_opts=opts.LegendOpts(pos_left="left"),
        datazoom_opts=[
            opts.DataZoomOpts(range_start=0, range_end=100),
            opts.DataZoomOpts(type_="inside", range_start=0, range_end=100),
        ],
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(name="水分", type_="value", max_=5),
    )
    .set_series_opts(
        markarea_opts=opts.MarkAreaOpts(
            is_silent=False,
            data=[
                opts.MarkAreaItem(
                    name="比例",
                    x=("2009/9/12-7:00", "2009/9/22-7:00"),
                    label_opts=opts.LabelOpts(is_show=False),
                    itemstyle_opts=opts.ItemStyleOpts(color="#DCA3A2", opacity=0.5),
                ),
                opts.MarkAreaItem(
                    name="水分",
                    x=("2009/9/10-7:00", "2009/9/20-7:00"),
                    label_opts=opts.LabelOpts(is_show=False),
                    itemstyle_opts=opts.ItemStyleOpts(color="#A1A9AF", opacity=0.5),
                ),
            ],
        ),
        axisline_opts=opts.AxisLineOpts(),
    )
    .render("数据评估4.6.html")
)

'/Users/zzm/Desktop/永信提供数据/数据评估4.6.html'

In [116]:
import math
error = []
adjustData = np.array(adjustData)
adjustdata = np.array(adjustdata)
for i in range(0,10000):
    res = adjustData[i] - adjustdata[i]
    error.append(math.fabs(res))
error,len(error)



([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [117]:
# list_new = []
# list_new.append(list(adjustdata))
# list_new.append(error)
# list_new

In [118]:

test = pd.DataFrame(data=error)
test.to_csv('error-4-6.csv',encoding='gbk')

In [42]:
test1 = pd.DataFrame(data=list(adjustdata))
test1.to_csv('predict.csv',encoding='gbk')